# Coding Outreach Group Summer Workshop
# Neuroimaging in Python

07/01/2021

**Content creators:** Elizabeth (Liz) Beard, Haroon Popal

**Content reviewers:** John Erardi

## Set Up & Prerequisites

## Description

## Outline

# Intro

#intro video

## Packages

### nilearn


### nltools


## Dowloading the Data

### Install Datalad

In [ ]:
# If you have anaconda
!conda install -c conda-forge datalad

# If you do not have anaconda, run the line below instead of the one above
#!pip install datalad

### Download data

In [ ]:
%cd ~/Desktop/datasets/

!datalad install https://gin.g-node.org/ljchang/Localizer

In [ ]:
 !brew install git-annex

### Map the data in python

In [5]:
import os
import glob
import nest_asyncio
nest_asyncio.apply()

import datalad.api as dl
import pandas as pd

In [6]:
localizer_path = '/Users/tuk12127/Desktop/datasets/Localizer'

dl.clone(source='https://gin.g-node.org/ljchang/Localizer', path=localizer_path)

Dataset('/Users/tuk12127/Desktop/datasets/Localizer')

In [7]:
ds = dl.Dataset(localizer_path)

In [8]:
results = ds.status(annex='all')

1794 annex'd files (0.0 B/42.1 GB present/total size)
nothing to save, working tree clean


In [9]:
file_list = glob.glob(os.path.join(localizer_path, '*', 'fmriprep', '*', 'func', '*tsv'))
file_list.sort()
file_list[:10]

['/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S01/func/sub-S01_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S02/func/sub-S02_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S03/func/sub-S03_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S04/func/sub-S04_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S05/func/sub-S05_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S06/func/sub-S06_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmriprep/sub-S07/func/sub-S07_task-localizer_desc-confounds_regressors.tsv',
 '/Users/tuk12127/Desktop/datasets/Localizer/derivatives/fmrip

In [10]:
result = ds.get(os.path.join(localizer_path, 'sub-S01'))
result = ds.get(glob.glob(os.path.join(localizer_path, '*.json')))
result = ds.get(glob.glob(os.path.join(localizer_path, '*.tsv')))
result = ds.get(glob.glob(os.path.join(localizer_path, 'phenotype')))

# Design Matrix

## 1. Load in onsets

## 2. Convolve with HRF

## 3. Nuisance covariates

## 4. Join onsets and covariates

## 5. Append runs

# First Level Analysis - Single Subject Data

## Load brain data

## Add design matrix

## Split odd and even runs

## Estimate model for all voxels

## Create contrast

## Visualization

# Second Level Analysis - Group Data

## Random-effects GLM
For all conditions

### Load first-level data for each subject

### Calculate the mean activation in each voxel across participants

### Perform one-sample t-test across all voxels

### Repeat for odd and even runs

## T-test
All conditions vs rest (fixation)

# Conclusion

#outro video